In [1]:
import pulp
import datagen as dgen
import organization as org
import itertools

def create_var_string(value):
    # Input in a 3-element-tuple
    return str(value[0])+'-'+str(value[1])+'-'+str(value[2])

# Get a list of teams
team_list = []
data = dgen.DataGen(2015)
conferences = data.league.conferences
for key in conferences.keys():
    conf = conferences[key]
    for div in conf.divisions.keys():
        teams = conf.divisions[div].teams
        for team in teams.keys():
            team_list.append(teams[team])

In [2]:
# Set number of games in the season
num_games = 82          
els = [tuple(x) for x in itertools.combinations(team_list, 2)]
# Create dictionary where keys are string representations of games
#    key: "{team1name}_{team2name}_{gamenum}"
#    value: information about home team and away team and game number
variable_dict = {}
for key in els:
    for idx in range(1,num_games+1):
        variable_dict[create_var_string((key[0].name,key[1].name,idx))] = {'home':key[0].name,'away':key[1].name,'homeDiv':key[0].division.name,\
                                                                           'homeConf':key[0].division.conference.name,\
                                                                           'awayConf':key[1].division.conference.name,\
                                                                           'awayDiv':key[1].division.name,'value':0,'gameNum':idx}
        variable_dict[create_var_string((key[1].name,key[0].name,idx))] = {'home':key[1].name,'away':key[0].name,'homeDiv':key[1].division.name,\
                                                                           'homeConf':key[1].division.conference.name,\
                                                                           'awayConf':key[0].division.conference.name,\
                                                                           'awayDiv':key[0].division.name,'value':0,'gameNum':idx}

x = pulp.LpVariable.dict("Game",variable_dict.keys(), lowBound=0, upBound=1, cat=pulp.LpInteger)
#Creating LP Problem
nba_schedule = pulp.LpProblem('NBA Schedule',pulp.LpMinimize)
for team in team_list:
    print("Adding conditions for ",team.name)
    nba_schedule += sum([x[key] for key in variable_dict.keys() if variable_dict[key]['home'] == team.name \
            or variable_dict[key]['away'] == team.name]) == num_games
    nba_schedule += sum([x[key] for key in variable_dict.keys() if variable_dict[key]['home'] == team.name])\
                         == num_games/2
    nba_schedule += sum([x[key] for key in variable_dict.keys() if variable_dict[key]['away'] == team.name])\
                         == num_games/2
    '''
    nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if (variable_dict[key]['home'] == team.name or\
                        variable_dict[key]['away'] == team.name) and \
                        variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv']]) == 16
    '''
    # Adding constraints for away and home games in a division
    for team_other in team_list:
        if team_other.name != team.name:
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) == 2
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['home'] == team.name and\
                            variable_dict[key]['away'] == team_other.name]) == 2
            #Out of conference teams
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeConf'] != variable_dict[key]['awayConf'] and\
                        variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) == 1
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeConf'] != variable_dict[key]['awayConf'] and\
                        variable_dict[key]['home'] == team.name and\
                            variable_dict[key]['away'] == team_other.name]) == 1
    # Ensuring one team plays only once on a single matchday       
    for num in range(1,num_games+1):
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                            if (variable_dict[key]['home'] == team.name or\
                                variable_dict[key]['away'] == team.name) and \
                            variable_dict[key]['gameNum'] == num]) == 1
    
    # Adding constraints for playing 4 games against the 6 selected out-of-division in-conference opponents
    
    teams_in_conf = set(team.division.conference.teams())
    assert(len(teams_in_conf) == 15)
    teams_in_div = set(team.division.teams.values())
    assert(len(teams_in_div) == 5)
    for team_other in team.conf_opponents:
        
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) == 2
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['home'] == team.name and \
                             variable_dict[key]['away'] == team_other.name]) == 2
    
    # Adding constraints 3 games against the 4 remaining out-of-division in-conference teams    
    remaining_in_conf_teams = teams_in_conf - teams_in_div - team.conf_opponents
    assert len(remaining_in_conf_teams) == 4
    for team_other in remaining_in_conf_teams:
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name]) <= 2
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if ((variable_dict[key]['away'] == team.name and \
                             variable_dict[key]['home'] == team_other.name) or (variable_dict[key]['home'] == team.name and \
                             variable_dict[key]['away'] == team_other.name))]) == 3

('Adding conditions for ', 'Thunder')
('Adding conditions for ', 'Jazz')
('Adding conditions for ', 'Timberwolves')
('Adding conditions for ', 'Nuggets')
('Adding conditions for ', 'Trailblazers')
('Adding conditions for ', 'Warriors')
('Adding conditions for ', 'Kings')
('Adding conditions for ', 'Lakers')
('Adding conditions for ', 'Clippers')
('Adding conditions for ', 'Suns')
('Adding conditions for ', 'Rockets')
('Adding conditions for ', 'Grizzlies')
('Adding conditions for ', 'Mavericks')
('Adding conditions for ', 'Spurs')
('Adding conditions for ', 'Pelicans')
('Adding conditions for ', 'Bucks')
('Adding conditions for ', 'Pistons')
('Adding conditions for ', 'Pacers')
('Adding conditions for ', 'Bulls')
('Adding conditions for ', 'Cavaliers')
('Adding conditions for ', 'Celtics')
('Adding conditions for ', 'Nets')
('Adding conditions for ', '76ers')
('Adding conditions for ', 'Raptors')
('Adding conditions for ', 'Knicks')
('Adding conditions for ', 'Magic')
('Adding conditio

In [ ]:

'''            
for key in variable_dict.keys():
    home = variable_dict[key]['home']
    away = variable_dict[key]['away']
    game_no = variable_dict[key]['gameNum']
    rev_key = create_var_string((away,home,game_no))
    nba_schedule += sum([x[key]+x[rev_key]]) <= 1

        for gameNum in range(1,num_games+1):    
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['home'] == team.name and\
                            variable_dict[key]['away'] == team_other.name\
                                and variable_dict[key]['gameNum'] == gameNum]) <= 1
'''

In [3]:
print("Solving")
nba_schedule.solve()

Solving


1

In [9]:
#Saving the LP object
nba_schedule.writeLP("nba_schedule.lp")

In [14]:
list_celtics = []
for key in variable_dict.keys():
    if (variable_dict[key]['home'] == 'Celtics' or variable_dict[key]['away'] == 'Celtics') and x[key].value() != 0:
         list_celtics.append((key,x[key].value(),variable_dict[key]['gameNum']))

In [15]:
sorted(list_celtics,key=lambda x: x[2])

[('Raptors-Celtics-1', 1.0, 1),
 ('76ers-Celtics-2', 1.0, 2),
 ('Celtics-Rockets-3', 1.0, 3),
 ('Heat-Celtics-4', 1.0, 4),
 ('Celtics-Knicks-5', 1.0, 5),
 ('Bucks-Celtics-6', 1.0, 6),
 ('Celtics-Nets-7', 1.0, 7),
 ('Cavaliers-Celtics-8', 1.0, 8),
 ('Hornets-Celtics-9', 1.0, 9),
 ('Celtics-Pistons-10', 1.0, 10),
 ('Celtics-Kings-11', 1.0, 11),
 ('Celtics-Magic-12', 1.0, 12),
 ('Celtics-Bulls-13', 1.0, 13),
 ('Celtics-Hawks-14', 1.0, 14),
 ('76ers-Celtics-15', 1.0, 15),
 ('Celtics-Nets-16', 1.0, 16),
 ('Celtics-Pacers-17', 1.0, 17),
 ('Magic-Celtics-18', 1.0, 18),
 ('Thunder-Celtics-19', 1.0, 19),
 ('Celtics-Pacers-20', 1.0, 20),
 ('Celtics-Raptors-21', 1.0, 21),
 ('Rockets-Celtics-22', 1.0, 22),
 ('Nets-Celtics-23', 1.0, 23),
 ('Warriors-Celtics-24', 1.0, 24),
 ('Knicks-Celtics-25', 1.0, 25),
 ('Timberwolves-Celtics-26', 1.0, 26),
 ('Cavaliers-Celtics-27', 1.0, 27),
 ('Celtics-Magic-28', 1.0, 28),
 ('Celtics-Heat-29', 1.0, 29),
 ('Celtics-Timberwolves-30', 1.0, 30),
 ('Pacers-Celtics-31

###Generating file to which is to be saved as json.

In [28]:
team_dict = {}
for team in team_list:
    team_dict[team.name] = team

In [106]:
schedule_dict={}
key_list = variable_dict.keys()
for key in key_list:
    if x[key].value() != 0:
        home = variable_dict[key]['home']
        away = variable_dict[key]['away']
        game_no = variable_dict[key]['gameNum']
        rev_key = create_var_string((away,home,game_no))
        key_list.remove(rev_key)
for key in key_list:
    if x[key].value() != 0:
        schedule_dict[(team_dict[variable_dict[key]['home']],variable_dict[key]['gameNum'])]\
        = (team_dict[variable_dict[key]['away']],'home')
        schedule_dict[(team_dict[variable_dict[key]['away']],variable_dict[key]['gameNum'])]\
        = (team_dict[variable_dict[key]['home']],'away')

In [114]:
#Saving the schedule directory
import json


TypeError: key (Team : Jazz, 70) is not a string

In [111]:
from schedule_tests import *
schedule_test(schedule_dict,82)

Test passed for: Nuggets
Test passed for: Timberwolves
Test passed for: Trailblazers
Test passed for: Warriors
Test passed for: Clippers
Test passed for: Suns
Test passed for: Kings
Test passed for: Lakers
Test passed for: Spurs
Test passed for: Mavericks
Test passed for: Grizzlies
Test passed for: Rockets
Test passed for: Pelicans
Test passed for: Raptors
Test passed for: Celtics
Test passed for: Knicks
Test passed for: Nets
Test passed for: 76ers
Test passed for: Cavaliers
Test passed for: Bulls
Test passed for: Pacers
Test passed for: Pistons
Test passed for: Bucks
Test passed for: Hawks
Test passed for: Heat
Test passed for: Hornets
Test passed for: Wizards
Test passed for: Magic
Test passed for: Thunder
Test passed for: Jazz
All tests passed


In [112]:
##Local copy of the function
def schedule_tests(schedule_dict,num_games):
    teams = set(map(lambda x:x[0],schedule_dict.keys()))
    teamNames = set(map(lambda x:x[0].name,schedule_dict.keys()))
    for team in teams:
        #print "Testing schedule for:",team.name
        home_count = 0
        away_count = 0
        home_cdiv = {}
        away_cdiv = {}
        total_cdiv = {}
        home_div = {}
        away_div = {}
        total_div = {}
        home_oconf = {}
        away_oconf = {}
        total_oconf = {}
        home_odiv = {}
        away_odiv = {}
        total_odiv = {}
        for conf_team in team.conf_opponents:
            home_cdiv[conf_team.name] = 0
            away_cdiv[conf_team.name] = 0
            total_cdiv[conf_team.name] = 0
        for div in team.division.teams:
            if div != team.name:
                home_div[div] = 0
                away_div[div] = 0
                total_div[div] = 0
        out_conf_set = set(teams) - set(team.division.conference.teams())
        for out_conf in out_conf_set:
            home_oconf[out_conf.name] = 0
            away_oconf[out_conf.name] = 0
            total_oconf[out_conf.name] = 0
        out_div_set = set(team.division.conference.teams()) - set(team.conf_opponents) - set(team.division.teams.values())
        for out_div in out_div_set:
            home_odiv[out_div.name] = 0
            away_odiv[out_div.name] = 0
            total_odiv[out_div.name] = 0
        #Running through all the games:
        for num in range(1,num_games+1):
            (opponent,g_type) = schedule_dict[(team,num)]
            
            if g_type == 'away':
                other = 'home'
            else:
                other = 'away'
            assert(schedule_dict[(opponent,num)] == (team,other))
            if g_type == 'home':
                home_count +=1
            else:
                away_count +=1
            if opponent in team.conf_opponents:
                if g_type == 'home':
                    home_cdiv[opponent.name] +=1
                    total_cdiv[opponent.name] +=1
                else:
                    away_cdiv[opponent.name] +=1
                    total_cdiv[opponent.name] +=1
            if opponent in out_conf_set:
                if g_type == 'home':
                    home_oconf[opponent.name] +=1
                    total_oconf[opponent.name] +=1
                else:
                    away_oconf[opponent.name] +=1
                    total_oconf[opponent.name] +=1
                    
            if opponent.name in team.division.teams:
                if g_type == 'home':
                    home_div[opponent.name] +=1
                    total_div[opponent.name] +=1
                else:
                    away_div[opponent.name] +=1
                    total_div[opponent.name] +=1
            
            if opponent in out_div_set:
                if g_type == 'home':
                    home_odiv[opponent.name] +=1
                    total_odiv[opponent.name] +=1
                else:
                    away_odiv[opponent.name] +=1
                    total_odiv[opponent.name] +=1
        
        #Checking out of conference:
        assert(list_compare(1, home_oconf.values()) == True)
        assert(list_compare(1, away_oconf.values()) == True)
        assert(list_compare(2, total_oconf.values()) == True)
        
        #Checking in 3 game teams:
        assert(list_compare(3, total_odiv.values()) == True)
        assert(all(i <= 2 for i in home_odiv.values()) == True)
        assert(all(i >= 1 for i in home_odiv.values()) == True)
        assert(all(i <= 2 for i in away_odiv.values()) == True)
        assert(all(i >= 1 for i in away_odiv.values()) == True)
        
        #Checking in division:
        assert(list_compare(2, home_div.values()) == True)
        assert(list_compare(2, away_div.values()) == True)
        assert(list_compare(4, total_div.values()) == True)
        
        #Checking in conference:
        assert(list_compare(2, home_cdiv.values()) == True)
        assert(list_compare(2, away_cdiv.values()) == True)
        assert(list_compare(4, total_cdiv.values()) == True)
        
        #Checking number of home and away games:
        assert(home_count == num_games/2)
        assert(away_count == num_games/2)
        print 'Test passed for:',team.name
    print 'All tests passed'

In [85]:
def list_compare(ref_value,list_check):
    for elem in list_check:
        if elem != ref_value:
            return False
    return True

###Testing on a smaller subset

In [185]:
league = {'A1':'A','A2':'A','A3':'A','A4':'A','B1':'B','B2':'B','B3':'B','B4':'B'}
els = [tuple(x) for x in itertools.combinations(league.keys(), 2)]
num_games = 12
team_list = league.keys()
variable_dict = {}
for key in els:
    for idx in range(1,num_games+1):
        variable_dict[create_var_string((key[0],key[1],idx))] = {'home':key[0],'away':key[1],'homeDiv':league[key[0]],\
                                                                           'awayDiv':league[key[1]],'value':0,'gameNum':idx}
        variable_dict[create_var_string((key[1],key[0],idx))] = {'home':key[1],'away':key[0],'homeDiv':league[key[1]],\
                                                                           'awayDiv':league[key[0]],'value':0,'gameNum':idx}

In [190]:
x = pulp.LpVariable.dict("Game",variable_dict.keys(), lowBound=0, upBound=1, cat=pulp.LpInteger)
#Creating LP Problem
nba_schedule = pulp.LpProblem('NBA Schedule',pulp.LpMinimize)
for team in team_list:
    print("Adding conditions for ",team)
    nba_schedule += sum([x[key] for key in variable_dict.keys() if variable_dict[key]['home'] == team \
            or variable_dict[key]['away'] == team]) == num_games
    for team_other in team_list:
        if team_other != team:
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['away'] == team and \
                             variable_dict[key]['home'] == team_other]) == 2
            
            nba_schedule += sum([x[key] for key in variable_dict.keys()\
                        if variable_dict[key]['homeDiv'] == variable_dict[key]['awayDiv'] and\
                        variable_dict[key]['home'] == team and\
                            variable_dict[key]['away'] == team_other]) == 2
            
            
    for num in range(1,num_games+1):
        nba_schedule += sum([x[key] for key in variable_dict.keys()\
                            if (variable_dict[key]['home'] == team or variable_dict[key]['away'] == team) and \
                            variable_dict[key]['gameNum'] == num]) == 1

('Adding conditions for ', 'A1')
('Adding conditions for ', 'B4')
('Adding conditions for ', 'A3')
('Adding conditions for ', 'A2')
('Adding conditions for ', 'A4')
('Adding conditions for ', 'B2')
('Adding conditions for ', 'B3')
('Adding conditions for ', 'B1')


In [192]:
nba_schedule.solve()

1

In [193]:
for key in variable_dict.keys():
    if (variable_dict[key]['home'] == 'A1' or variable_dict[key]['away'] == 'A1') and x[key].value() != 0:
        print (key,x[key].value())

('A1-A3-11', 1.0)
('A4-A1-9', 1.0)
('A3-A1-8', 1.0)
('A2-A1-1', 1.0)
('A2-A1-6', 1.0)
('A1-A4-2', 1.0)
('A1-A3-4', 1.0)
('A4-A1-10', 1.0)
('A3-A1-12', 1.0)
('A1-A2-5', 1.0)
('A1-A2-3', 1.0)
('A1-A4-7', 1.0)


In [33]:
team_list[0].division.conference.teams()

[Team : Thunder,
 Team : Jazz,
 Team : Timberwolves,
 Team : Nuggets,
 Team : Trailblazers,
 Team : Warriors,
 Team : Kings,
 Team : Lakers,
 Team : Clippers,
 Team : Suns,
 Team : Rockets,
 Team : Grizzlies,
 Team : Mavericks,
 Team : Spurs,
 Team : Pelicans]